In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import os
import re

(注意)：在此之前，需要将前面两部分得到的数据交叉匹配，结果存放在match中，接下来开始这一步

# 将匹配表按信噪比snrg分为多个文件，并分别下载链接
Dividing the APOGEE-LAMOST cross-match table into intervals according to the signal-to-noise ratio

## 读文件

In [5]:
# match = pd.read_csv(r'G:\Star\1_Data_download_and_preprocessing\raw_data_catalogue_label\DR11_match_denoise_50-200.csv')
match = pd.read_csv(r'G:\Star\1_Data_download_and_preprocessing\raw_data_catalogue_label\match_denoise_100-200_1arc.csv')

In [6]:
match.columns

Index(['obsid_1', 'uid_1', 'snrg_1', 'class_1', 'z_1', 'ra_1', 'dec_1',
       'obsid_2', 'uid_2', 'snrg_2', 'class_2', 'z_2', 'ra_2', 'dec_2',
       'GroupID', 'GroupSize', 'Separation'],
      dtype='object')

## 定义拆分函数

In [7]:

# folder_path = r'E:\my_star\download\snrg_split'  # 定义保存CSV文件的文件夹路径
folder_path = r'./download/snrg_split_match_catalog'  # 定义保存CSV文件的文件夹路径
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    

def depart_low(i,j):

    snrg=match.iloc[np.where((match["snrg_1"]>=i)&(match["snrg_1"]<j))]

    print("snrg_{}_{}".format(i,j)," len:",len(snrg))

    snrg.to_csv(os.path.join(folder_path, "match_low_{}_{}.csv".format(i,j))) 

    return snrg

def depart_high(i,j):

    snrg=match.iloc[np.where((match["snrg_2"]>=i)&(match["snrg_2"]<j))]

    print("snrg_{}_{}".format(i,j)," len:",len(snrg))

    snrg.to_csv(os.path.join(folder_path, "match_high_{}_{}.csv".format(i,j))) 

    return snrg

In [8]:
def depart2_high(i):
    snrg=match.iloc[np.where(match["snrg_2"]>=i)]
    print("snrg_{}".format(i)," len:",len(snrg))
    snrg.to_csv(os.path.join(folder_path, "match_high_{}.csv".format(i))) 
    return snrg

## 进行拆分

In [9]:
# 将低信噪比数据分为5组
for i in range(5):
    depart_low(10*i,10*(i+1))

snrg_0_10  len: 29899
snrg_10_20  len: 48786
snrg_20_30  len: 47143
snrg_30_40  len: 49269
snrg_40_50  len: 51967


In [10]:
# 将高信噪比数据分为3组
depart_high(100,120)
depart_high(120,200)
# depart2_high(200)

snrg_100_120  len: 111476
snrg_120_200  len: 115588
snrg_200  len: 0


,obsid_1,uid_1,snrg_1,class_1,z_1,ra_1,dec_1,obsid_2,uid_2,snrg_2,class_2,z_2,ra_2,dec_2,GroupID,GroupSize,Separation


# 生成obsid 文件

这个文件作为下一步下载的目录

In [11]:
directory_path =  "./download/snrg_split_match_catalog"
save_path =  "./download/download_obsid"
if not os.path.exists(save_path):
    os.makedirs(save_path)
    



# directory_path =  r'E:\my_star\download\snrg_split'
# save_path =  r'E:\my_star\download\download_obsid'

file_list = os.listdir(directory_path)

# 遍历文件列表
for filename in file_list:
    file_path= os.path.join(directory_path, filename)  # 当前文件的path
    # 检查是否为文件（而非子目录）
    if os.path.isfile(file_path):
        save_filename = re.sub(r'\.csv$', '.txt', filename)
        save_file_path = os.path.join(save_path,save_filename)
        catalog = pd.read_csv(file_path) 
        

        if 'low' in save_file_path:
            
            observed_ids = set()        
            # 打开文件写入数据
            with open(save_file_path, 'w') as file:
                # 遍历DataFrame的每行
                for obsid in catalog['obsid_1']:
                    # 检查obsid是否已经在集合中
                    if obsid not in observed_ids:
                        # 如果不在集合中，写入文件并添加到集合中
                        file.write(f'{obsid}\n')
                        observed_ids.add(obsid)  # 将当前obsid添加到集合中，表示已处理
                        
        if 'high' in save_file_path:
            
            observed_ids = set()        
            # 打开文件写入数据
            with open(save_file_path, 'w') as file:
                # 遍历DataFrame的每行
                for obsid in catalog['obsid_2']:
                    # 检查obsid是否已经在集合中
                    if obsid not in observed_ids:
                        # 如果不在集合中，写入文件并添加到集合中
                        file.write(f'{obsid}\n')
                        observed_ids.add(obsid)  # 将当前obsid添加到集合中，表示已处理                
